In [ ]:
import pandas as pd
import re
import ast  # A library to safely evaluate string literals


def generate_stimuli_js_from_csv(
    csv_path: str, num_samples: int, modality: str = "both"
):
    """
    Reads a stimulus CSV file and generates a JavaScript array for the experiment.

    Args:
        csv_path (str): The file path to your CSV file.
        num_samples (int): The number of samples (rows) to process from the top of the file.
        modality (str): The experiment type. Can be 'text', 'image', or 'both'.
                        This determines which data is included in the output.
    """
    try:
        df = pd.read_csv(csv_path)
    except FileNotFoundError:
        print(f"Error: The file '{csv_path}' was not found.")
        return

    # Ensure we don't request more samples than available
    if num_samples > len(df):
        print(
            f"Warning: Requested {num_samples} samples, but only {len(df)} are available. Using all available samples."
        )
        num_samples = len(df)

    df_subset = df.head(num_samples).copy()

    # --- Data Cleaning and Processing ---

    # 1. Extract the true_value (ratio) from the image_path using regex
    def extract_ratio(path):
        # This regex looks for a number like 0.123 in a path like '..._0.123.png'
        match = re.search(r"_(\d+\.\d+)\.png$", str(path))
        return float(match.group(1)) if match else "N/A"

    df_subset["true_value"] = df_subset["image_path"].apply(extract_ratio)

    # 2. Parse the ascii_line column
    def parse_ascii(line_str):
        try:
            # ast.literal_eval safely evaluates the string "('line1', 'line2')" into a tuple
            lines_tuple = ast.literal_eval(line_str)
            # Join the two lines with a newline character
            return "\n".join(lines_tuple)
        except (ValueError, SyntaxError):
            # Handle cases where the format might be incorrect
            return "Error parsing ASCII line"

    df_subset["parsed_ascii"] = df_subset["ascii_line"].apply(parse_ascii)

    # --- Generate the JavaScript Code ---

    js_objects = []
    for _, row in df_subset.iterrows():
        # Escape backticks inside the ascii art itself to avoid breaking the JS template literal
        js_safe_ascii = row["parsed_ascii"].replace("`", "\\`")

        obj_parts = [f"        type: '{modality}'"]
        obj_parts.append(f"        true_value: {row['true_value']}")

        if modality in ["text", "both"]:
            obj_parts.append(f"        ascii_art: `{js_safe_ascii}`")
        if modality in ["image", "both"]:
            # Prepend 'images/' to the path, assuming the script is one level up from the images folder
            image_path = f"images/{row['image_path'].split('/')[-1]}"
            obj_parts.append(f"        image_path: '{image_path}'")

        js_objects.append("    {\n" + ",\n".join(obj_parts) + "\n    }")

    final_js_code = "const stimuli = [\n" + ",\n".join(js_objects) + "\n];"

    print("✅ JavaScript 'stimuli' array generated successfully!")
    print(
        "📋 Copy the entire code block below and paste it into your 'script.js' file."
    )
    print("-" * 70)
    print(final_js_code)
    print("-" * 70)

In [ ]:
# --- HOW TO USE ---
# 1. Make sure your CSV file is in the same directory as your notebook, or provide the full path.
# 2. Adjust the file name and number of samples as needed.
# 3. Choose the modality: 'text', 'image', or 'both'.

# Example for your first file (10 samples, both image and text)
exp_type = "text_image"
exp_name = "line_length_ratio"

base_path = f"../../experiments/{exp_type}/{exp_name}/data/experiment_files"
range_cat = "0.1_0.5"
data_path = f"{base_path}/{range_cat}/line_len_ratio_{range_cat}_100samples.csv"
csv_file = f"{base_path}/{range_cat}/line_len_ratio_{range_cat}_100samples.csv"

generate_stimuli_js_from_csv(csv_path=csv_file, num_samples=10, modality="both")

# Example for an image-only experiment
# generate_stimuli_js_from_csv(csv_path=csv_file, num_samples=10, modality='image')

# Example for a text-only experiment
# generate_stimuli_js_from_csv(csv_path=csv_file, num_samples=10, modality='text')

✅ JavaScript 'stimuli' array generated successfully!
📋 Copy the entire code block below and paste it into your 'script.js' file.
----------------------------------------------------------------------
const stimuli = [
    {
        type: 'both',
        true_value: 0.2498,
        ascii_art: `|-=-=-----                               |
|-------------------------.------ -------|`,
        image_path: 'images/sample_000_0.2498.png'
    },
    {
        type: 'both',
        true_value: 0.4803,
        ascii_art: `|------------.-~----                     |
|-------------.~-------------------------|`,
        image_path: 'images/sample_001_0.4803.png'
    },
    {
        type: 'both',
        true_value: 0.3928,
        ascii_art: `|-----~---=-----                         |
|---------------------------=------~-----|`,
        image_path: 'images/sample_002_0.3928.png'
    },
    {
        type: 'both',
        true_value: 0.3395,
        ascii_art: `|----------~-                           

In [9]:
import pandas as pd
import re
import ast


def build_full_experiment_js(experiment_plan: list):
    """
    Builds a complete, multi-block JavaScript stimuli array from a plan.

    Args:
        experiment_plan (list): A list of dictionaries, where each dictionary
                                defines a block of the experiment.
    """
    full_stimuli_list = []

    # --- Helper function to process a single CSV ---
    def process_csv_for_block(config: dict):
        csv_path = config["csv_path"]
        num_samples = config["num_samples"]
        modality = config["modality"]
        block_name = config["block_name"]

        try:
            df = pd.read_csv(csv_path).head(num_samples)
        except FileNotFoundError:
            print(f"ERROR: File not found -> {csv_path}")
            return None

        # --- Data processing ---
        def extract_ratio(path):
            match = re.search(r"_(\d+\.\d+)\.png$", str(path))
            return float(match.group(1)) if match else "N/A"

        def parse_ascii(line_str):
            try:
                lines_tuple = ast.literal_eval(line_str)
                return "\n".join(lines_tuple)
            except (ValueError, SyntaxError):
                return "Error"

        df["true_value"] = df["image_path"].apply(extract_ratio)
        df["parsed_ascii"] = df["ascii_line"].apply(parse_ascii)

        block_stimuli = []
        for _, row in df.iterrows():
            stimulus = {
                "type": modality,
                "block_name": block_name,
                "true_value": row["true_value"],
                "ascii_art": row["parsed_ascii"],
                "image_path": f"images/{row['image_path'].split('/')[-1]}",
            }
            block_stimuli.append(stimulus)
        return block_stimuli

    # --- Build the full experiment from the plan ---
    for i, block_config in enumerate(experiment_plan):
        # Add an instruction screen before each block (except the very first one)
        if i > 0:
            instruction_stimulus = {
                "type": "instruction",
                "title": "Next Block",
                "text": f"You have completed the previous block. The next set of trials will now begin.",
            }
            full_stimuli_list.append(instruction_stimulus)

        block_data = process_csv_for_block(block_config)
        if block_data:
            full_stimuli_list.extend(block_data)

    # --- Generate the final JavaScript Code ---
    js_objects = []
    for stimulus in full_stimuli_list:
        obj_parts = [f"        type: '{stimulus['type']}'"]
        if stimulus["type"] == "instruction":
            obj_parts.append(f"        title: '{stimulus['title']}'")
            obj_parts.append(f"        text: '{stimulus['text']}'")
        else:  # It's a trial
            obj_parts.append(f"        block_name: '{stimulus['block_name']}'")
            obj_parts.append(f"        true_value: {stimulus['true_value']}")
            if stimulus["type"] in ["text", "both"]:
                # Escape backticks inside the ascii art itself
                js_safe_ascii = stimulus["ascii_art"].replace("`", "\\`")
                obj_parts.append(f"        ascii_art: `{js_safe_ascii}`")
            if stimulus["type"] in ["image", "both"]:
                obj_parts.append(f"        image_path: '{stimulus['image_path']}'")

        js_objects.append("    {\n" + ",\n".join(obj_parts) + "\n    }")

    final_js_code = "const stimuli = [\n" + ",\n".join(js_objects) + "\n];"

    print("✅✅✅ Complete multi-block experiment code generated! ✅✅✅")
    print(
        "📋 Copy the entire code block below and paste it into the NEW 'script.js' file."
    )
    print("-" * 70)
    print(final_js_code)
    print("-" * 70)


# ==============================================================================
# --- HOW TO USE: DEFINE YOUR FULL EXPERIMENT PLAN HERE ---
# ==============================================================================
# Create a list of dictionaries. Each dictionary is one "block" of your experiment.
# You can add as many blocks as you want, in any order.

# ASSUMING YOU HAVE 3 CSV FILES FOR THE 3 RANGES
# Let's pretend they are named: 'range1.csv', 'range2.csv', 'range3.csv'
# Replace with your actual file names.

exp_type = "text_image"
exp_name = "line_length_ratio"

base_path = f"../../experiments/{exp_type}/{exp_name}/data/experiment_files"
range_cat_list = ["0.1_0.5", "0.3_0.8", "0.5_0.9"]
data_path = f"{base_path}/{range_cat}/line_len_ratio_{range_cat}_100samples.csv"
csv_file = f"{base_path}/{range_cat}/line_len_ratio_{range_cat}_100samples.csv"


experiment_plan = []

for range_cat in range_cat_list:
    # Add the text only tests

    experiment_plan.append(
        {
            "block_name": f"Text-Only, Range {range_cat}",
            "csv_path": f"{base_path}/{range_cat}/line_len_ratio_{range_cat}_100samples.csv",
            "num_samples": 10,
            "modality": "text",
        }
    )

for range_cat in range_cat_list:
    # Add the image only tests

    experiment_plan.append(
        {
            "block_name": f"Image-Only, Range {range_cat}",
            "csv_path": f"{base_path}/{range_cat}/line_len_ratio_{range_cat}_100samples.csv",
            "num_samples": 10,
            "modality": "image",
        }
    )

for range_cat in range_cat_list:
    # Add the image only tests

    experiment_plan.append(
        {
            "block_name": f"Text + Image, Range {range_cat}",
            "csv_path": f"{base_path}/{range_cat}/line_len_ratio_{range_cat}_100samples.csv",
            "num_samples": 10,
            "modality": "both",
        }
    )

# Run the function to generate the code
build_full_experiment_js(experiment_plan)

✅✅✅ Complete multi-block experiment code generated! ✅✅✅
📋 Copy the entire code block below and paste it into the NEW 'script.js' file.
----------------------------------------------------------------------
const stimuli = [
    {
        type: 'text',
        block_name: 'Text-Only, Range 0.1_0.5',
        true_value: 0.2498,
        ascii_art: `|-=-=-----                               |
|-------------------------.------ -------|`
    },
    {
        type: 'text',
        block_name: 'Text-Only, Range 0.1_0.5',
        true_value: 0.4803,
        ascii_art: `|------------.-~----                     |
|-------------.~-------------------------|`
    },
    {
        type: 'text',
        block_name: 'Text-Only, Range 0.1_0.5',
        true_value: 0.3928,
        ascii_art: `|-----~---=-----                         |
|---------------------------=------~-----|`
    },
    {
        type: 'text',
        block_name: 'Text-Only, Range 0.1_0.5',
        true_value: 0.3395,
        ascii_art: